In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist

import string

from proyecto_ampliacion import Proyecto_ampliacion
from funciones_extra import *

In [ ]:
ej1 = "El proyecto consiste en el aumento de capacidad de la subestación Cañete mediante la instalación de un nuevo transformador 66/23 kV y al menos 16 MVA de capacidad con Cambiador de Derivación Bajo Carga (CDBC), y sus respectivos paños de conexión en ambos niveles de tensión. A su vez, el proyecto incluye la ampliación de la barra e instalaciones comunes del patio de 66 kV de la subestación, cuya configuración corresponde a barra simple, la cual deberá considerar espacio en barra y plataforma para dos posiciones, de manera de permitir la conexión del nuevo transformador de poder antes mencionado y la normalización de la conexión de la línea 1x66 kV Tres Pinos – Cañete. Adicionalmente, el proyecto considera la construcción de una nueva sección de barra de 23 kV, en configuración barra simple, contemplándose la construcción de, al menos, dos paños para alimentadores, el paño de conexión del transformador antes mencionado, un paño para la conexión de un banco de condensadores y la construcción de un paño de interconexión con la barra existente. En caso de definirse el desarrollo de la ampliación de este patio como una sala de celdas, se deberán considerar los paños contenidos en esta descripción junto con la construcción de una celda para equipos de medida, la construcción de una celda para servicios auxiliares si corresponde. Además, el proyecto contempla la instalación de dos nuevos bancos de condensadores de 3,6 MVAr en 23 kV, de dos etapas cada uno, conectados a cada sección de barra de 23 kV con sus respectivos paños de conexión."


In [ ]:

def separar_por_patios(texto):
    parrafos = texto.split(".")
    palabras_clave_patios = ["patio"]
    palabras_clave_celdas = ["sala de celdas"]
    palabras_clave_paños = ["paño", "paños"]
    palabras_posiciones = ["diagonales", "posiciones", "posición", "diagonal"]
    parrafos_patios = [parrafo for parrafo in parrafos if any(palabra in parrafo for palabra in palabras_clave_patios) and "configuración" in parrafo and any(palabra in parrafo for palabra in palabras_posiciones)]
    parrafos_celdas = [parrafo for parrafo in parrafos if any(palabra in parrafo for palabra in palabras_clave_celdas) and "configuración" in parrafo and any(palabra in parrafo for palabra in palabras_posiciones)]
    parrafos_paños = [parrafo for parrafo in parrafos if any(palabra in parrafo for palabra in palabras_clave_paños) and "configuración" and any(palabra in parrafo for palabra in palabras_posiciones)]
    parrafos_otros = [parrafo for parrafo in parrafos if parrafo not in parrafos_patios and parrafo not in parrafos_celdas and parrafo not in parrafos_paños]
    
    diccionario_parrafos = {
        "patios": parrafos_patios,
        "celdas": parrafos_celdas,
        "paños": parrafos_paños,
        "otros": parrafos_otros
    }

    return diccionario_parrafos
    

In [ ]:
def remove_stopwords(texto):
    stop_words = set(stopwords.words('spanish'))
    tokens = nltk.word_tokenize(texto)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [word for word in tokens if word.lower() not in string.punctuation]
    return " ".join(tokens)

In [44]:
def clasificar_parrafo(parrafo):
    tipo_aumento_capacidad = ["aumento capacidad", "instalación nuevo transformador", "reemplazo actual transformador"] #caso inst o const trafo
    tipo_ampliacion_construccion = ["ampliación barra", "construcción nueva sección barra", "ampliación galpón", "ampliación patio", "construcción nueva barra", "ampliación sala celdas", "construcción nueva sala celdas", "construcción nuevo paño"] # caso ampliacion construccion patio, paño, nva barra, etc
    tipo_otro = ["nuevos bancos condensadores", "nuevo banco condensadores", "banco autotransformadores existente nueva"]

    tipo_no_interesa = ["proyecto incluye todas obras modificaciones", "respectivas bases licitación", "contempla todas tareas labores obras", "caso definirse desarrollo", " producto del aumento de capacidad antes", "Entrada operación", "Valor inversión", "C.O.M.A", "referenciales V.I", "referencial proyecto"]

    tipo = "REVISAR!"
    elemento_encontrado = None

    #quizas, podria devolver directamente el elemento_encontrado y definir las listas en la otra funcion, tal que si elemento encontrado esta en la lista "tipo_aumento_capacidad", se utiliza el esquema de estilo xml correspondiente al aumento de capacidad de la s/e, y asi con los otros tipos de parrafos.

    for elemento in tipo_aumento_capacidad:
        if elemento in parrafo:
            tipo = "construccion_instalacion_trafo"
            elemento_encontrado = elemento
            return tipo, elemento_encontrado
    
    for elemento in tipo_ampliacion_construccion:
        if elemento in parrafo:
            tipo = "ampliacion_construccion_patio"
            elemento_encontrado = elemento
            return tipo, elemento_encontrado
    
    for elemento in tipo_otro:
        if elemento in parrafo:
            tipo = "otro"
            elemento_encontrado = elemento
            return tipo, elemento_encontrado
    
    for elemento in tipo_no_interesa:
        if elemento in parrafo:
            tipo = "no_interesa"
            elemento_encontrado = elemento
            return tipo, None


    return tipo, elemento_encontrado
    
    


In [ ]:
parrafos = sent_tokenize(ej1)


for parrafo in parrafos:
    parrafo_limpio = remove_stopwords(parrafo)
    tipo = clasificar_parrafo(parrafo_limpio)
    print(f"Parrafo tipo {tipo}: {parrafo}")
    print("\n")


In [43]:
file = "plan_expansion_final_2023.pdf"
dic_amp = generar_diccionario_ampliaciones(file)
dic_desc_amp = generar_diccionario_descripciones_amp(file, dic_amp)


for titulo, descripcion in dic_desc_amp.items():
    print(titulo, "\n")
    parrafos = sent_tokenize(descripcion)
    for parrafo in parrafos:
        parrafo_limpio = remove_stopwords(parrafo)
        tipo, elemento_encontrado = clasificar_parrafo(parrafo_limpio)

        if tipo == "REVISAR!":
            print(f"Parrafo tipo {tipo}: {parrafo_limpio}")
            print(f"Elemento encontrado que define TIPO en esquema: {elemento_encontrado}")

        elif tipo == "no_interesa":
            print(f"Parrafo tipo {tipo}: {parrafo}")
            print(f"Elemento encontrado que define TIPO en esquema: {elemento_encontrado}")

        else:
            print(f"Parrafo tipo {tipo}: {parrafo}")
            print(f"Elemento encontrado que define TIPO en esquema: {elemento_encontrado}")
    
    print("\n")

Ampliación en S/E Cóndores 110 kV (BP+BT)  

Parrafo tipo ampliacion_construccion_patio: Descripción general y ubicación de la obra El proyecto consiste en la ampliación de las barras e instalaciones comunes del patio de 110 kV de la subestación Cóndores, cuya configuración corresponde a barra principal con barra de transferencia, para dos nuevas posiciones, de manera de permitir la conexión de la obra “Nueva S/E Alto Molle y nueva línea 2x110 kV Alto Molle – Cóndores”.
Elemento encontrado que define TIPO en esquema: ampliación barra
Parrafo tipo no_interesa: El proyecto incluye todas las obras, modificaciones y labores necesarias para la ejecución y puesta en servicio de las nuevas instalaciones, tales como adecuaciones en los patios respectivos, adecuación de las protecciones, comunicaciones, SCADA, obras civiles, montaje, malla de puesta a tierra y pruebas de los nuevos equipos, entre otras.
Elemento encontrado que define TIPO en esquema: proyecto incluye todas obras modificaciones
